<a href="https://colab.research.google.com/github/jasonwillschiu/scratch/blob/master/meetup%20bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
# this way doesn't let us download it to memory so we can open and read it
# we either download as string which can't read the pickle
# or we download to file which adds an extra step
# from google.cloud import storage
# from io import StringIO

# set this environ variable to get storage.Client() auth'ed
# the trick here is going to the bucket and adding the service account to it
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "robotic-tiger-270605-60df8bc892bb.json"
# client = storage.Client()
# bucket = client.get_bucket('meetup-script')
# fileList = [x.name for x in list(client.list_blobs(bucket))]
# fileList

In [65]:
# this way gives us a small library (not tensorflow)
# that can open files from gcs
# to be used in cloud run, gcsfs isn't supported by cloud functions

import gcsfs
import pickle
# need to upload the json cred for google colab
fs = gcsfs.GCSFileSystem(project='robotic-tiger-270605',token='robotic-tiger-270605-60df8bc892bb.json')
print(fs.ls('meetup-script'))
with fs.open(fs.ls('meetup-script')[0],'rb') as f:
  print(pickle.load(f))

['meetup-script/meetup_cookies_chrome_2020-09-19.pkl']
[{'domain': '.meetup.com', 'httpOnly': True, 'name': 'MEETUP_SESSION', 'path': '/', 'sameSite': 'None', 'secure': True, 'value': 'id=3944202b-8b58-4d5e-899f-d9dda5b23150'}, {'domain': '.meetup.com', 'expiry': 1601910980, 'httpOnly': False, 'name': '_uetvid', 'path': '/', 'secure': False, 'value': 'bd94336af3fa22803f34a4be867604a0'}, {'domain': '.meetup.com', 'expiry': 1607764578, 'httpOnly': True, 'name': 'MEETUP_SEGMENT', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'member'}, {'domain': '.meetup.com', 'httpOnly': True, 'name': 'MEETUP_LANGUAGE', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'language=en&country=AU'}, {'domain': '.meetup.com', 'expiry': 1600593380, 'httpOnly': False, 'name': '_uetsid', 'path': '/', 'secure': False, 'value': 'fe71119e40b43b7294e7a632fed0e3ad'}, {'domain': '.meetup.com', 'httpOnly': False, 'name': 'MUP_jqueryEn', 'path': '/', 'sameSite': 'Lax', 'secure': True, 'value': 'on'}, 

In [67]:
# can run these in Dockerfile
# selenium, gcsfs and chromium install
!pip install gcsfs
!pip install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 3.0MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension adobe-flashplugin
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 11 not upgraded.
Need to get 79.2 MB of archives.
After this operation, 268 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 85.0.4183.83-0ubuntu0.18.04.2 [1,118 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 85.0.4183.83-0ubuntu0.18.04.2 [70.3 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-up

In [5]:
# authenticate for colab to look at gcs with tensorflow
# from google.colab import auth
# auth.authenticate_user()
# print("auth'ed for colab")

auth'ed for colab


In [71]:
from time import sleep
# use gcsfs to read files from gcs
import gcsfs
import pickle
from selenium import webdriver

# the group to look inside
#TEST_URL   = 'https://www.meetup.com/en-AU/Fishburners-Meetup/events/'
TOP_SHOT_EVENTS_URL = 'https://www.meetup.com/Topshot-Badminton-Meetup/events/'

# cookie location local
#cookie_name = 'meetup_cookies_chrome_2020-09-19.pkl'

# cookie location on gcs
# cookie_gcs = 'gs://meetup-script/meetup_cookies_chrome_2020-09-19.pkl'
fs = gcsfs.GCSFileSystem(project='robotic-tiger-270605',token='robotic-tiger-270605-60df8bc892bb.json')
# print(fs.ls('meetup-script')) is a list of files in that bucket


# picking the chrome driver
OPTIONS = webdriver.ChromeOptions()
# remove all these when watching it the first time
OPTIONS.add_argument('-headless') # no browser
OPTIONS.add_argument('-no-sandbox') # not a test browser
OPTIONS.add_argument('-disable-dev-shm-usage') # so docker will allocate >64MB to chrome
# if we need to be a realer browser (not a headless chrome)
OPTIONS.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""')
driver = webdriver.Chrome('chromedriver',options=OPTIONS)
print('chromedriver - loaded')

# load the page without the cookie
driver.get('https://meetup.com')
print('meetup.com - loaded without cookie')
# load the cookie from google cloud storage
with fs.open(fs.ls('meetup-script')[0],'rb') as f:
  cookies = pickle.load(f)  
for cookie in cookies:
    driver.add_cookie(cookie)
# load the page again after the cookie is loaded, this is the same as logging in
driver.get('https://meetup.com')
print('meetup.com - loaded with cookie')

# go to our meetup group
driver.get(TOP_SHOT_EVENTS_URL)
print(f'meetup group {TOP_SHOT_EVENTS_URL} - loaded')

# find events to attend, this will be in a funky format
elem = driver.find_elements_by_partial_link_text("Attend")
# grab the actual URLs and remove the ?action bit
elems = []
for e in elem[:4]:
  # remove the action rsvp yes part, because that doesn't seem to work (19 Sep 2020)
  elems.append(e.get_attribute('href').split('?action')[0])

# try attend first 4 events in the list
# hence for topshot having 3x sessions a week
# 4th sessions will always be the new open one
for e in elems[:4]:
  # go to the first event page
  driver.get(e)
  print(f'meetup group event - {e} loaded')

  try: 
    # click attend button, but this won't always exist
    driver.find_element_by_class_name('gtmEventFooter--attend-btn').click()
    print('meetup group event - clicked attend')

  except Exception as e:
    print(f'no attend button to click with error: \n{e}')

  # wait 1/2 a second before trying another
  sleep(0.5)
  print('-----wait 1/2 a second then try next')

chromedriver - loaded
meetup.com - loaded without cookie
meetup.com - loaded with cookie
meetup group https://www.meetup.com/Topshot-Badminton-Meetup/events/ - loaded
meetup group event - https://www.meetup.com/en-AU/Topshot-Badminton-Meetup/events/xqmcvrybcmbkc/ loaded
meetup group event - clicked attend
-----wait 1/2 a second then try next
meetup group event - https://www.meetup.com/en-AU/Topshot-Badminton-Meetup/events/wflktrybcmbmc/ loaded
no attend button to click with error: Message: no such element: Unable to locate element: {"method":"css selector","selector":".gtmEventFooter--attend-btn"}
  (Session info: headless chrome=85.0.4183.83)

-----wait 1/2 a second then try next
meetup group event - https://www.meetup.com/en-AU/Topshot-Badminton-Meetup/events/tnpqtrybcnbcb/ loaded
no attend button to click with error: Message: no such element: Unable to locate element: {"method":"css selector","selector":".gtmEventFooter--attend-btn"}
  (Session info: headless chrome=85.0.4183.83)

-